In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### 벡터 DB 생성

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

# /home/wanted-1/potenup-workspace/Project/Final_Project/team2/Unreal_AI/apps/voice_quest/data/PDF/Epilouge(final).pdf
loader = PyMuPDFLoader("./data/PDF/Epilouge(final).pdf")
docs = loader.load()

def preprocess(text):
    new_text = text.replace("Document", "")
    return new_text

for i in range(3):
    docs[i].page_content = preprocess(docs[i].page_content)

In [7]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

def split_with_bracket_keyword(documents):
    result = []
    pattern = r"\[(.*?)\]\n"  # [키워드] 패턴

    for doc in documents:
        text = doc.page_content
        matches = list(re.finditer(pattern, text))

        for i, match in enumerate(matches):
            keyword = match.group(1)
            start = match.end()
            end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
            chunk_text = text[start:end].strip()

            if chunk_text:
                result.append(
                    Document(
                        page_content=chunk_text,
                        metadata={**doc.metadata, "keyword": keyword}
                    )
                )
    return result

# keyword 기준으로 먼저 쪼개고
split_docs = split_with_bracket_keyword(docs)

# 그 다음에 chunk 나누기
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n", " "]  # 여기선 ### 안 써도 됨, 위에서 이미 쪼갬
)

chunks = splitter.split_documents(split_docs)

# 결과 확인
for chunk in chunks:
    print("---")
    print(f"keyword: {chunk.metadata.get('keyword')}")


---
keyword: 달빛 연못
---
keyword: 달빛 연못
---
keyword: 연잎밥
---
keyword: 연잎밥
---
keyword: 삶은 당근
---
keyword: 개구리 동산
---
keyword: 연못 파리
---
keyword: 왕왕벌


In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding = OpenAIEmbeddings()
vector_db = Chroma.from_documents(documents=chunks, embedding=embedding, persist_directory="./data/chroma_db", collection_name="openai")
vector_db.persist()


/tmp/ipykernel_1013506/284930069.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


### 퀴즈 LLM

In [10]:
# 퀴즈 정답 랜덤으로 선택
import random

answer_list = ["달빛 연못", "연잎밥", "삶은 당근", "개구리 동산", "연못 파리", "왕왕벌"]
answer = random.choice(answer_list)
print(answer)

달빛 연못


In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [12]:

template = """
You are a mom.  
Please follow the rules to create a natural and warm question for your child, based on the correct answer.

Rules:  
- Answer in Korean.  
- The question must be only one sentence.  
- Never directly mention the correct answer.  
- Use emotional or visual cues from the context to help the child recall the memory naturally.  
- Do not explain or give hints. Only create a question.  
- Use casual and friendly Korean language.

Answer: {answer}

context: {context}

question:
"""

prompt = ChatPromptTemplate.from_template(template)

vector_db = Chroma(persist_directory="./data/chroma_db", embedding_function=OpenAIEmbeddings(), collection_name="openai")

retriever = vector_db.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k" : 2, "filter" : {"keyword" : answer}}
)

llm = ChatGoogleGenerativeAI(model= "gemini-2.0-flash", temperature=0.5)

chain = (
    {"context" : itemgetter("answer") | retriever, "answer" : itemgetter("answer")}
    | prompt
    | llm 
    | StrOutputParser()
)

response = chain.invoke({"answer" : answer})
print(response)


/tmp/ipykernel_1013506/4192657665.py:22: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory="./data/chroma_db", embedding_function=OpenAIEmbeddings(), collection_name="openai")


우리 아기가 처음으로 물에 발 담글 때, 연못 위에 뭐가 예쁘게 비쳐서 반짝거렸더라?


### 힌트 생성 LLM

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema.messages import HumanMessage

In [13]:
template = """
너는 다정하고 따뜻한 엄마야. 아이가 퀴즈를 풀고 있는데, 정답을 직접 알려주면 안 돼.  
아이 스스로 기억해낼 수 있도록 감성적이고 비유적인 힌트를 주는 게 너의 역할이야.

규칙:
- 힌트는 2~3문장으로 구성해야 해.
- 정답을 직접 언급하거나, 글자 수, 철자, 초성, 위치 등을 알려주는 식의 단서는 주지 마.
- 특히 아래 단어들은 힌트에서 절대 사용하지 마: ["연", "잎", "밥"]
- 아이가 질문을 해도 항상 힌트로만 답해. 어떤 질문에도 정답의 직접적인 정보는 주지 않아.
- 아이가 오답을 말하면 "그건 아니야~"처럼 부드럽게 반응하고, 이어서 힌트를 자연스럽게 줘.
- 말투는 일상적인 엄마 말투로, 너무 문어체처럼 딱딱하게 말하지 않아.

입력:
- 아이의 답변: {user_word}
- 정답: {answer}
- 대화 기록: {chat_history}
- 참고 문맥: {context}

출력:
- 엄마의 힌트:
"""

prompt = ChatPromptTemplate.from_template(template)

vector_db = Chroma(persist_directory="./data/chroma_db", embedding_function=OpenAIEmbeddings(), collection_name="openai")

retriever = vector_db.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k" : 2, "filter" : {"keyword" : answer}}
)

llm = ChatGoogleGenerativeAI(model= "gemini-2.0-flash", temperature=0.5)

chat_history = []

chain = (
    {
        "context": lambda x: retriever.invoke(x["user_word"].content),
        "user_word": itemgetter("user_word"),   
        "answer": itemgetter("answer"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

history_store = {}  # 세션 기록을 저장할 딕셔너리

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    print(session_id)
    if session_id not in history_store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        history_store[session_id] = ChatMessageHistory()
    return history_store[session_id]  # 해당 세션 ID에 대한 세션 기록 반환

history_chain = RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
            chain,  # 실행할 Runnable 객체
            get_session_history,  # 세션 기록을 가져오는 함수
            input_messages_key="user_word",  # 입력 메시지의 키
            history_messages_key="chat_history",  # 기록 메시지의 키
        )
response = history_chain.invoke(
    {
        "user_word": HumanMessage(content="정답이 몇 글자야?"),  # ← 여기를 HumanMessage로!
        "answer": answer
    },
    config={"configurable": {"session_id": "Jacode97"}}
)
print(response)

Jacode97
우리 개구락이가 먹었던 특별한 음식 기억나니? 
음, 잎사귀의 향긋함이 밥알 하나하나에 스며들어 있었지. 마치 네 걱정이 스르륵 녹아내리는 것처럼 말이야. 


In [14]:
response = history_chain.invoke(
    {
        "user_word": HumanMessage(content="정답이 몇글자냐고"),  # ← 여기를 HumanMessage로!
        "answer": answer
    },
    config={"configurable": {"session_id": "Jacode97"}}
)
print(response)

Jacode97
아이고, 궁금했쪄? 
음... 개구락이가 연못에서 처음 맛본 특별한 음식 있잖아. 넓고 푸른 잎에 싸여서 향긋함이 가득했던 그 밥 말이야. 마치 네가 좋아하는 꽃처럼 귀한 재료들이 밥 속에 숨어 있었지.


### 음성 데이터 base64 복원

In [13]:
import sounddevice as sd
from scipy.io.wavfile import write

def record_wav(filename="./data/audio/output.wav", duration=5, samplerate=44100):
    print("녹음 시작...")
    recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()  # 녹음이 끝날 때까지 대기
    write(filename, samplerate, recording)
    print(f"녹음 완료: {filename}")

record_wav()

녹음 시작...


FileNotFoundError: [Errno 2] No such file or directory: './data/audio/output.wav'

In [64]:
# 테스트용 데이터 생성하기
import base64

def wav_to_base64(wav_path: str) -> str:
    """WAV 파일을 base64 문자열로 변환"""
    with open(wav_path, "rb") as wav_file:
        wav_bytes = wav_file.read()
    base64_str = base64.b64encode(wav_bytes).decode("utf-8")
    return base64_str

def split_base64_chunks(base64_data, chunk_size):
    total_size = len(base64_data)
    chunks = [base64_data[i:i+chunk_size] for i in range(0, total_size, chunk_size)]
    return total_size, len(chunks), chunks

wav_path = "./data/audio/output.wav"
chunk_size = 1024  # 청크 크기 조절 가능

# base64 문자열 변환
base64_audio = wav_to_base64(wav_path)

# base64를 청크 나누기
total_size, num_chunks, chunks = split_base64_chunks(base64_audio, chunk_size)
print(chunks[0])
print(num_chunks)

UklGRsy6BgBXQVZFZm10IBAAAAABAAEARKwAAIhYAQACABAAZGF0Yai6BgAAAAAA//8AAAAAAAAAAP//AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP//AAAAAAAAAAAAAAAAAAAAAAAAAQAAAP//AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAD//wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAEAAAAAAAAAAQABAAAAAAAAAP//AQABAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAABAAAAAAAAAAAAAAD//wAAAAAAAAAAAAAAAAAAAAAAAAAA////////AAAAAAAAAAD//wAAAAAAAAAAAAAAAAAAAAAAAAEAAAABAAAAAAAAAAAAAAAAAAEA//8AAAEA//8AAAAAAAAAAAAAAAAAAP//AAAAAAAA//8AAP//AAAAAAAAAAAAAP//AAAAAAAAAAAAAAAA//8AAAAAAAAAAAAAAAAAAAAA/////wAAAAAAAAAAAAABAAAAAAAAAAAAAAD//wAAAAAAAAAAAAAAAAAAAAAAAAEAAAABAAAAAAABAP//AQAAAAAAAAD/////AAAAAAAA//8AAP//AAAAAAAA//8AAAAAAAAAAAEAAAAAAAAAAAAAAAAAAQAAAAEAAAAAAAAAAAAAAAAAAQAAAAAAAAABAAEAAAAAAAAAAAAAAAAAAAAAAP//AQAAAAAAAAAAAAAAAAAAAAEAAAAAAP//AAAAAAAA/////wAAAAABAAAA//8AAAAAAAAAAAAAAAD/////AAAAAP//AQAAAAEAAAAAAAAAAAAAAAAAAQAAAAAAAAD//wAAAAAAAAAAAAABAP//AAAAAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAD//wAAAAD//wAAAAD//wAAAAAAAAAAAAAAAAAAAAABAAAAAAD//wEA

In [65]:
buffers = {}  
total_sizes = {}  
chunk_sizes = {} 

def init_buffer(quiz_id, total_size, chunk_size):
    buffers[quiz_id] = {}
    total_sizes[quiz_id] = total_size
    chunk_sizes[quiz_id] = chunk_size

def receive_packet(quiz_id, start, index, fin, total_size, chunk_size, raw_data):
    """패킷 받으면서 복원 시도"""
    global buffers, total_sizes, chunk_sizes

    if start == 1:
        print(f"[{quiz_id}] 새 전송 시작. 초기화.")
        init_buffer(quiz_id, total_size, chunk_size)

    buffers[quiz_id][index] = raw_data

    if fin == 1:
        print(f"[{quiz_id}] 마지막 패킷 받음. 복원 시작.")
        chunks = buffers[quiz_id]
        sorted_data = b''.join(chunks[i] for i in sorted(chunks.keys()))
        # 버퍼 정리
        del buffers[quiz_id]
        del total_sizes[quiz_id]
        del chunk_sizes[quiz_id]
        return sorted_data
    return None

In [ ]:
reconstructed_data = None
quiz_id = "재식스 퀴즈"

for index, chunk in enumerate(chunks):
    start = 1 if index == 0 else 0
    fin = 1 if index == num_chunks - 1 else 0
    reconstructed_data = receive_packet(
        quiz_id=quiz_id,
        start=start,
        index=index,
        fin=fin,
        total_size=total_size,
        chunk_size=chunk_size,
        raw_data=chunk.encode("utf-8")  # 문자열 → bytes (여기가 중요!)
    )
    if reconstructed_data:
        print(f"복원 성공띄~! 복원된 크기: {len(reconstructed_data)} bytes")

reconstructed_str = reconstructed_data.decode("utf-8")
decoded_bytes = base64.b64decode(reconstructed_str)
with open("./data/wav/restore.wav", "wb") as f:
    f.write(decoded_bytes)
assert reconstructed_str == base64_audio, "망했습니다. 데이터가 원본과 달라용~!"

### STT(Speech To Text)

In [18]:
import numpy as np
import io
import soundfile as sf 
import whisper
import librosa 

#Whisper 모델 로드
model = whisper.load_model("medium")  # tiny, base, small, medium, large 중 선택

In [67]:
# base64 문자열 → 바이너리 디코드
decoded_wav_bytes = base64.b64decode(reconstructed_data)

# BytesIO로 감싸서 읽기
audio_bytes = io.BytesIO(decoded_wav_bytes)

# WAV → numpy array
audio_data, samplerate = sf.read(audio_bytes, dtype='float32')

print(f"샘플레이트: {samplerate}, 오디오 길이: {len(audio_data)}")

# Whisper가 16kHz를 요구하니까 필요하면 리샘플링
if samplerate != 16000:
    audio_data = librosa.resample(audio_data, orig_sr=samplerate, target_sr=16000)
    samplerate = 16000

# STT 수행
result = model.transcribe(audio_data, fp16=False, language='ko')
print(result["text"])

샘플레이트: 44100, 오디오 길이: 220500
 개구리 쌈박


### 모듈화

In [68]:
class WavReconstructor:
    '''쪼개진 wav파일 복원 클래스'''
    def __init__(self):
        self.buffers = {}
        self.total_sizes = {}
        self.chunk_sizes = {}

    def init_buffer(self, quiz_id, total_size, chunk_size):
        '''새 전송 시작 시 해당 quiz_id의 버퍼 초기화'''
        self.buffers[quiz_id] = {}
        self.total_sizes[quiz_id] = total_size
        self.chunk_sizes[quiz_id] = chunk_size

    def receive_packet(self, quiz_id, start, index, fin, total_size, chunk_size, raw_data):
        '''
        start가 1일때 전송 시작 및 버퍼 초기화,
        fin이 1일때 패킷 join 후 return 
        '''
        if start == 1:
            # print(f"[{quiz_id}] 새 전송 시작. 초기화.")
            self.init_buffer(quiz_id, total_size, chunk_size)

        self.buffers[quiz_id][index] = raw_data

        if fin == 1:
            # print(f"[{quiz_id}] 마지막 패킷 받음. 복원 시작.")
            chunks = self.buffers[quiz_id]
            sorted_data = b''.join(chunks[i] for i in sorted(chunks.keys()))
            # 버퍼 정리
            del self.buffers[quiz_id]
            del self.total_sizes[quiz_id]
            del self.chunk_sizes[quiz_id]
            return sorted_data
        return None

In [69]:
sock = WavReconstructor()
# print(total_size, num_chunks, chunks)
for index, chunk in enumerate(chunks):
    start = 1 if index == 0 else 0
    fin = 1 if index == num_chunks - 1 else 0
    reconstructed_data = sock.receive_packet(
        quiz_id=quiz_id,
        start=start,
        index=index,
        fin=fin,
        total_size=total_size,
        chunk_size=chunk_size,
        raw_data=chunk.encode("utf-8") 
    )
    if reconstructed_data:
        print(f"복원 성공띄~! 복원된 크기: {len(reconstructed_data)} bytes")

##########SAVE POINT###############
reconstructed_str = reconstructed_data.decode("utf-8")
decoded_bytes = base64.b64decode(reconstructed_str)
with open("./data/audio/output.wav", "wb") as f:
    f.write(decoded_bytes)
assert reconstructed_str == base64_audio, "망했습니다. 데이터가 원본과 달라용~!"

복원 성공띄~! 복원된 크기: 588060 bytes


In [70]:
class STTEngine:
    '''Speech to Text 클래스(Whisper모델 사용)'''
    def __init__(self):
        self.model = whisper.load_model("medium")
    
    def base_to_np(self, base64_str):
        '''base64를 numpy로 변환하는 메서드'''
        wav_bytes = base64.b64decode(base64_str)
        audio_bytes = io.BytesIO(wav_bytes)
        audio, samplerate = sf.read(audio_bytes, dtype="float32")
        if samplerate != 16000:
            # print(f"샘플레이트가 16,000Hz가 아닙니다!, 샘플{samplerate}Hz, 오디오{len(audio)}")
            audio = librosa.resample(audio, orig_sr=samplerate, target_sr=16000)
            samplerate = 16000
            # print(f"리샘플 후 {samplerate}Hz로 리샘플")
        return audio
    
    def stt(self, base64_str):
        result = self.model.transcribe(self.base_to_np(base64_str), fp16=False, language="ko")
        # print(f"[STT Text] {result['text']}")

        return result['text']

In [71]:
stt = STTEngine()
user_word = stt.stt(reconstructed_data)

In [72]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from operator import itemgetter
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema.messages import HumanMessage

from dotenv import load_dotenv
load_dotenv()

True

In [73]:
class QuestLLM:
    def __init__(self, prompt_path, db_params, answer):
        self.llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.5)
        self.prompt_path = prompt_path
        self.answer = answer
        self.vector_db = Chroma(**db_params)
        self.prompt = self._set_prompt()
        self.retriever = self._make_retriever()

    def _make_retriever(self):
        retriever = self.vector_db.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 2, "filter": {"keyword": self.answer}}
        )
        return retriever

    def _get_template(self):
        with open(self.prompt_path, "r", encoding="utf-8") as f:
            return f.read()

    def _set_prompt(self):
        return ChatPromptTemplate.from_template(self._get_template())

    def _make_chain(self):
        pass

In [74]:

class Quiz(QuestLLM):
    '''트리거 시작하면 퀴즈를 내는 LLM'''
    def __init__(self, prompt_path, db_params, answer):
        super().__init__(prompt_path, db_params, answer)
        self.chain = self._make_chain()

    def _make_chain(self):
        chain = (
            {
                "context": itemgetter("answer") | self.retriever,
                "answer": itemgetter("answer"),
            }
            | self.prompt
            | self.llm
            | StrOutputParser()
        )
        return chain

    def start_quiz(self):
        return self.chain.invoke({"answer": self.answer})

In [ ]:
answer = "연잎밥"
db_params = {
    "persist_directory": "./data/chroma_ver2",
    "embedding_function" : OpenAIEmbeddings(),
    "collection_name": "openai"
}
prompt_path = "./prompts/quiz.prompt"
quiz = Quiz(prompt_path,db_params, answer)
quiz.start_quiz()

'우리 개구락이, 엄마랑 연못가에서 잎사귀에 싸서 먹었던 밥, 무슨 향이 났었는지 기억나?'

In [24]:
from pydantic import BaseModel, Field 

class AnswerEvaluation(BaseModel):
    is_answer : bool = Field(..., description="Please tell me if it is correct(True/False).")
    hint : str = Field("", description="Generate a hint")

In [82]:
from langchain_core.output_parsers import PydanticOutputParser

class Hint(QuestLLM):
    def __init__(self, prompt_path, db_params, answer):
        super().__init__(prompt_path, db_params, answer)
        self.history_store = {}
        self.chain = self._make_chain()

    def get_session_history(self, session_id):
        if session_id not in self.history_store:
            self.history_store[session_id] = ChatMessageHistory()
        return self.history_store[session_id]

    def _make_chain(self):
        chain = (
            {
                "context": lambda x: self.retriever.invoke(x["user_word"].content),
                "user_word": itemgetter("user_word"),   
                "answer": itemgetter("answer"),
                "chat_history": itemgetter("chat_history"),
            }
            | self.prompt
            | self.llm
            | StrOutputParser()
        )

        return RunnableWithMessageHistory(
            chain,
            self.get_session_history,
            input_messages_key="user_word",
            history_messages_key="chat_history",
        )

    def similarity(self, user_word):
        """user_word와 self.answer의 유사도(float)만 리턴"""
        embedding_model = OpenAIEmbeddings()
        user_vec = np.array(embedding_model.embed_query(user_word))
        answer_vec = np.array(embedding_model.embed_query(self.answer))

        similarity = np.dot(user_vec, answer_vec) / (np.linalg.norm(user_vec) * np.linalg.norm(answer_vec))
        return similarity
    
    def contain_word(self,user_word):
        contain = self.answer in user_word
        return contain
    
    def invoke(self, user_word, session_id="default"):
        response = self.chain.invoke(
            {
                "user_word": HumanMessage(content=user_word),
                "answer": self.answer
            },
            config={"configurable": {"session_id": session_id}}
        )
        # print(f"[힌트] : {response}")
        similarity = self.similarity(user_word)
        contain = self.contain_word(user_word)
        return {
            "contain": contain,
            "response" : response,
            "similarity" : similarity
        }

In [ ]:
prompt_path = "./prompts/hint.prompt"
hint = Hint(prompt_path, db_params, answer)
response = hint.invoke("개구리 쌈밥")

[힌트] : 아이고, 우리 아가 틀렸네. 😓 음... 엄마가 맑은 연못에서 푸르고 커다란 잎으로 정성스럽게 싸서 지어줬던 밥 기억나? 잣이랑 은행도 넣고 말이야. 😌


In [51]:
type(response['response'])

str

In [90]:
hint.invoke("연잎밥")

Error in RootListenersTracer.on_chain_end callback: ValueError("Expected str, BaseMessage, list[BaseMessage], or tuple[BaseMessage]. Got is_answer=False hint='아이고, 아쉽다! 음... 개구락이가 밥을 먹으면서 연못을 바라봤잖아. 그때 연못 위에 뭐가 잔잔히 떠 있었더라? 그리고 엄마가 그 잎에 밥을 곱게 싸줬었지.'.")


{'is_answer': False,
 'hint': '아이고, 아쉽다! 음... 개구락이가 밥을 먹으면서 연못을 바라봤잖아. 그때 연못 위에 뭐가 잔잔히 떠 있었더라? 그리고 엄마가 그 잎에 밥을 곱게 싸줬었지.',
 'similarity': np.float64(0.9328349416930161)}

In [76]:
import re

def text_processing(text):
    new_text = re.sub(r"[^가-힣a-zA-Z0-9\s.,!?]", "", text)
    return new_text

In [77]:
def tts(text,index):
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.VoiceSelectionParams(
        language_code="ko-KR",
        name="ko-KR-Standard-B",
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config,
    )

    # The response's audio_content is binary.
    with open(f"./data/audio/output_{index}.mp3", "wb") as out:
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')


In [78]:
db_params = {
    "persist_directory": "./data/chroma_db",
    "embedding_function" : OpenAIEmbeddings(),
    "collection_name": "openai"
}
stt = STTEngine()

In [ ]:
import random
answer_list = ["달빛 연못", "연잎밥", "취당근", "개를리랄로 게를랄라 산", "말왕대벌", "깝파리"]
answer = random.choice(answer_list)
print(f"[정답] {answer}")

# 1. 퀴즈 생성
quiz = Quiz(
    prompt_path="./prompts/quiz.prompt",
    db_params=db_params,
    answer = answer
)
print(f"[엄마(퀴즈)] : {quiz.start_quiz()}")

# 2. 사용자 답변 전처리

sock = WavReconstructor()
# print(total_size, num_chunks, chunks)
for index, chunk in enumerate(chunks):
    start = 1 if index == 0 else 0
    fin = 1 if index == num_chunks - 1 else 0
    reconstructed_data = sock.receive_packet(
        quiz_id=quiz_id,
        start=start,
        index=index,
        fin=fin,
        total_size=total_size,
        chunk_size=chunk_size,
        raw_data=chunk.encode("utf-8") 
    )

# 3. STT 변환
user_word = stt.stt(reconstructed_data)

# 4. 힌트 생성
hint = Hint(
    prompt_path="./prompts/hint.prompt",
    db_params=db_params,
    answer = answer
)
# 5. TTS 변환
result = text_processing(response['response'])
tts(result,1)

print(f"[개구리] : {user_word}")
response = hint.invoke(user_word)
print(f"[엄마(힌트)] : {response['response']}")
print(f"유사도 : {response['similarity']}, 글자수 : {response['response'][0]}")

[정답] 연잎밥
[엄마(퀴즈)] : 음~ 그때 연못가에서 엄마가 잎으로 예쁘게 싸줬던 밥, 무슨 향이 났었지?
Audio content written to file "output.mp3"
[개구리] :  개구리 쌈박
[힌트] : 아이고, 땡은 아니야~ 엄마가 정성스럽게 싸줬던 밥 기억나니? 향긋한 잎에 싸여서 은은한 향이 났었잖아. 
[엄마(힌트)] : 아이고, 땡은 아니야~ 엄마가 정성스럽게 싸줬던 밥 기억나니? 향긋한 잎에 싸여서 은은한 향이 났었잖아. 
유사도 : 0.8312653295074122, 글자수 : 아
